# Modeling and Evaluation

## Objectives
Predict medical insurance charges using customer profile information.

## Inputs
- Processed customer dataset with feature engineering.

## Outputs
- Trained ML regression model.
- Feature importance ranking.

# Change working directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Engineered Dataset

In [ ]:
import pandas as pd
df_path = 'outputs/datasets/engineered/insurance_engineered.csv'
df = pd.read_csv(df_path)
df.head()

Stop displaying warnings messages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# ML Pipeline with all data

## ML pipeline for Feature Engineering

In [ ]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from feature_engine.encoding import OneHotEncoder
from feature_engine.selection import SmartCorrelatedSelection

def PipelineDataCleaningAndFeatureEngineering():
    # Define the pipeline for data cleaning and feature engineering
    pipeline_base = Pipeline([
        ('onehot_encoding', OneHotEncoder(drop_last=True)),
        ('correlated_selection', SmartCorrelatedSelection(
            method='pearson',
            threshold=0.6,
            selection_method='variance'))
    ])
    
    return pipeline_base

PipelineDataCleaningAndFeatureEngineering()

---

## ML Pipeline for Modelling and Hyperparameter Optimisation

In [ ]:
# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# Define a pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

def PipelineClf(model):
    pipeline_base = Pipeline([
        ("scaler", StandardScaler()),
        ("feat_selection", SelectFromModel(model)),
        ("model", model),
    ])

    return pipeline_base

**Hyperparameter Optimisation**

This is the process of tuning the hyperparameters of a machine learning model to improve its performance. It involves searching for the best combination of hyperparameters that yield the highest performance on a validation set.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
import numpy as np

class HyperparameterOptimizationSearch(BaseEstimator):
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineClf(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

## Split Train and Test Set

In [ ]:
# Features (X) and Target (y)
X = df.drop('charges', axis=1)
y = df['charges']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2)

print(f"X_train shape: {X_train.shape}  y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}  y_test shape: {y_test.shape}")

## Grid Search CV - Sklearn

In [ ]:

models_quick_search = {
    "LinearRegression": LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    "LinearRegression": {},
    "DecisionTreeRegressor": {
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2]
    },
    "RandomForestRegressor": {
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2],
        'model__bootstrap': [True]
    },
    "GradientBoostingRegressor": {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.1, 0.2],
        'model__max_depth': [3, 5],
        'model__subsample': [0.8, 1.0],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2]
    },
    "XGBRegressor": {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.1, 0.2],
        'model__max_depth': [3, 5],
        'model__subsample': [0.8, 1.0],
        'model__colsample_bytree': [0.8, 1.0]
    },
}

**Run Grid Search CV**

In [ ]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Grid Search
search = HyperparameterOptimizationSearch(
    models=models_quick_search,
    params=params_quick_search
)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

Check the results

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
print(grid_search_summary)

**Evaluate the Best Model**

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)

def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, prediction)).round(3))
    print("\n")

best_model = grid_search_summary.iloc[0]['estimator']
print("Best Model:", best_model)
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_

regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)

**Do an extensive search on the most suitable algorithm to find the best hyperparameter configuration.**

In [ ]:
models_search = {
    "XGBRegressor": XGBRegressor(random_state=0, objective='reg:squarederror'),
}

params_search = {
    "XGBRegressor": {
        'model__n_estimators': [100, 300],
        'model__learning_rate': [0.1, 0.05, 0.01],
        'model__max_depth': [3, 6, 10],
        'model__subsample': [0.8, 1.0],
        'model__colsample_bytree': [0.8, 1.0]
    }
}


In [ ]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

search = HyperparameterOptimizationSearch(
    models=models_search,
    params=params_search
)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)


In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

We can see the best score is achieved by the `XGBRegressor` extensive search is lower then the `RandonForestRegressor`. 

We will run the extensive search on `RandonForestRegressor` model to compare the scores.

In [ ]:
models_search = {
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
}

params_search = {
    "RandomForestRegressor": {
        'model__n_estimators': [100, 300],
        'model__max_depth': [10, 20, None],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4],
        'model__bootstrap': [True, False]
    }
}

search = HyperparameterOptimizationSearch(
    models=models_search,
    params=params_search
)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

**According to the scores results, RandomForestRegressor seem to be the best model for this dataset**

In [ ]:
best_model = grid_search_summary.iloc[0,0]
best_model

Parameters for best model

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

The best clf pipeline

In [ ]:
pipeline_clf = grid_search_pipelines[best_model].best_estimator_
pipeline_clf

---

## Assess feature importance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get selected features from the pipeline
selected_features = X_train.columns[pipeline_clf.named_steps['feat_selection'].get_support()]
importances = pipeline_clf.named_steps['model'].feature_importances_

# Create importance dataframe
df_feature_importance = (
    pd.DataFrame({'Feature': selected_features, 'Importance': importances})
    .sort_values(by='Importance', ascending=False)
)

# Save ordered list of best features
best_features = df_feature_importance['Feature'].tolist()

# Print top features
print(f"* These are the {len(best_features)} most important features in descending order. ")
print(df_feature_importance)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=df_feature_importance, x='Importance', y='Feature', palette='viridis')
plt.title("Feature Importance - RandomForestRegressor")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.tight_layout()
# plt.savefig("outputs/ml_pipeline/v1/feature_importance_rf.png", dpi=300)
plt.show()



---

## Evaluate Pipeline on Train and Test Sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

def regression_evaluation(X, y, pipeline):
    prediction = pipeline.predict(X)
    print('R2 Score:', r2_score(y, prediction).round(3))
    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))
    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, prediction)).round(3))
    print("\n")

def regression_performance(X_train, y_train, X_test, y_test, pipeline):
    print("Model Evaluation \n")
    print("* Train Set")
    regression_evaluation(X_train, y_train, pipeline)
    print("* Test Set")
    regression_evaluation(X_test, y_test, pipeline)


In [ ]:
regression_performance(X_train, y_train, X_test, y_test, pipeline_clf)


We used R2 Score, MAE and RMSE to evaluate the model performance, since Confusion Matrix and Accuracy are not suitable for regression problems.

**Evaluation Conclusion**

The Random Forest Regressor, demonstrates strong predictive performance and generalization capability in estimating medical insurance costs. Its test set R² score of 0.841 indicates that the model explains approximately 84.1% of the variance in insurance charges for unseen data.
The model achieved a Mean Absolute Error (MAE) of approximately 2892, meaning that on average, its predictions deviate from the true insurance charges by about $2,892, which is acceptable given the typical range of medical costs. Additionally, the Root Mean Squared Error (RMSE) of around $4,880 confirms that the model performs well without being heavily skewed by large outliers.

# Push files to Repo

We will generate the following file
* Train set
* Test set
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* features importance plot

In [ ]:
import joblib
import os

version = "v1"
file_path = f"outputs/ml_pipelines/{version}"

try:
    os.makedirs(file_path)
except Exception as e:
    print(e)

## Train Set

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

## Test Set

In [ ]:
print(X_test.shape)
X_test.head()

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

## ML Pipelines: Feature Engineering and Modelling

Pipeline responsible for Data Cleaning and Feature Engineering.

In [ ]:
pipeline_data_cleaning_feat_eng = PipelineDataCleaningAndFeatureEngineering()
pipeline_data_cleaning_feat_eng

In [ ]:
joblib.dump(value=pipeline_data_cleaning_feat_eng ,
            filename=f"{file_path}/clf_pipeline_data_cleaning_feat_eng.pkl")

Pipeline responsible for Feature Scaling, and Model

In [ ]:
pipeline_clf

In [ ]:
joblib.dump(value=pipeline_clf ,
            filename=f"{file_path}/clf_pipeline_model.pkl")

## Feature Importance Plot

In [ ]:
df_feature_importance.plot(kind='bar',x='Feature',y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar', x='Feature', y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')